In [3]:
# import des bibliothèques requises
import os
import openai
import requests

# données d'identification et d'authentification openAI
openai.organization = "org-n6X2zCb7o31ARXwZnVkEeopQ"
openai.api_key = os.getenv("OPENAI_API_KEY")


In [72]:
# le moteur IA utilisé dans ce programme
engine = "text-davinci-002"

# la liste suivante est ici hardcodée mais sera générée depuis une liste d'ingrédients suggérés par l'interface utilisateur (frontend)
ingredients_from_UI = ["beef","ginger","bell peppers","oregano"]

# à partir de la liste d'ingrédients, génération d'une chaîne de caractères qui s'intègrera naturellement dans la question posée
formatted_ingredients = "\n- " + "\n- ".join(ingredients_from_UI)

# la question qui sera posée au modèle openAI
question_asked_to_openAI_completions = "Provide a cooking recipe based on the following ingredients:\n" + formatted_ingredients + "\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions."

# un dictionnaire contenant les arguments à passer à l'endpoint "completions" via la méthode "openai.Completion.create" (SDK openAI)
completion_arguments = {
  "engine" : engine,
  "prompt" : question_asked_to_openAI_completions,
  "temperature" : 0.7,
  "top_p" : 1,
  "max_tokens" : 300,
  "frequency_penalty" : 0,
  "presence_penalty" : 0
  }

print(completion_arguments)


{'engine': 'text-davinci-002', 'prompt': "Provide a cooking recipe based on the following ingredients:\n\n- beef\n- ginger\n- bell peppers\n- oregano\n\nThe response should include the recipe's name, the recipe's ingredients, and the recipe's instructions.", 'temperature': 0.7, 'top_p': 1, 'max_tokens': 300, 'frequency_penalty': 0, 'presence_penalty': 0}


In [17]:
# requête openAI paramétrée

response = openai.Completion.create(
  engine = completion_arguments["engine"],
  prompt =  completion_arguments["prompt"],
  temperature = completion_arguments["temperature"],
  top_p = completion_arguments["top_p"],
  max_tokens = completion_arguments["max_tokens"],
  frequency_penalty = completion_arguments["frequency_penalty"],
  presence_penalty = completion_arguments["presence_penalty"])


In [68]:
complete_recipe = response["choices"][0]["text"][2:]

recipe_name = complete_recipe[21:complete_recipe.find("\n")-1]

ingredients_extraction = complete_recipe[complete_recipe.find("Ingredients:") + 14 : complete_recipe.find("Instructions:")-2]
instructions_extraction = complete_recipe[complete_recipe.find("Instructions:") + 15:]

ingredients_list = list(ingredients_extraction.split('\n'))
ingredients_list = list(i.strip('- ') for i in ingredients_list)

instructions_list = list(instructions_extraction.split('\n'))
instructions_list = list(i[3:] for i in instructions_list)

print("Nom de la recette : \n" + recipe_name)
print("Liste d'ingrédients : \n" + "\n".join(ingredients_list))
print("Recette : \n" + "\n".join(instructions_list))

Nom de la recette : 
Beef and Peppers with Oregano
Liste d'ingrédients : 
1 pound beef, thinly sliced
1 teaspoon ginger, minced
2 bell peppers, thinly sliced
1 teaspoon oregano, dried
1 tablespoon olive oil
salt and pepper to taste
Recette : 
In a large skillet over medium-high heat, add the beef and ginger. Cook until the beef is browned.
Add the bell peppers and oregano. Cook for 3-4 minutes, until the peppers are slightly softened.
Drizzle with olive oil and season with salt and pepper to taste. Serve immediately.


In [90]:
# définition de la fonction qui trouvera une recette en fonction d'ingrédients fournis
# la fonction ne prend pour l'instant qu'un seul argument

def findrecipe(ingredients, temperature):
    response = openai.Completion.create(
        engine = completion_arguments["engine"],
        prompt =  completion_arguments["prompt"],
        temperature = temperature,
        top_p = completion_arguments["top_p"],
        max_tokens = completion_arguments["max_tokens"],
        frequency_penalty = completion_arguments["frequency_penalty"],
        presence_penalty = completion_arguments["presence_penalty"])

    complete_recipe = response["choices"][0]["text"][2:]

    recipe_name = complete_recipe[:complete_recipe.find("\n")-1]

    ingredients_extraction = complete_recipe[complete_recipe.find("Ingredients:") + 14 : complete_recipe.find("Instructions:")-2]
    instructions_extraction = complete_recipe[complete_recipe.find("Instructions:") + 15:]

    ingredients_list = list(ingredients_extraction.split('\n'))
    ingredients_list = list(i.strip('- ') for i in ingredients_list)

    instructions_list = list(instructions_extraction.split('\n'))
    instructions_list = list(i[3:] for i in instructions_list)

    print("=========================================\n"+str(response)+"\n=========================================\n")

    return {
        "RecipeName" : recipe_name,
        "Ingredients" : ingredients_list,
        "Instructions" : instructions_list
    }


In [91]:
# appel de la fonction

findrecipe(ingredients_from_UI,.7)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nBeef and Bell Pepper Stir-Fry Recipe\n\nIngredients:\n- 1 lb beef (sirloin, flank, or round steak), thinly sliced\n- 1/2 teaspoon ginger\n- 2 bell peppers (red, yellow, or green), julienned\n- 1 teaspoon oregano\n\nInstructions:\n1. In a large skillet or wok, stir-fry beef in ginger over medium-high heat until browned.\n2. Add bell peppers and oregano and stir-fry until peppers are crisp-tender.\n3. Serve with rice or noodles."
    }
  ],
  "created": 1665154020,
  "id": "cmpl-5yiuqM15ZWBcLRyZSqotmZ0FpwNKF",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 129,
    "prompt_tokens": 49,
    "total_tokens": 178
  }
}



{'RecipeName': 'Beef and Bell Pepper Stir-Fry Recip',
 'Ingredients': ['1 lb beef (sirloin, flank, or round steak), thinly sliced',
  '1/2 teaspoon ginger',
  '2 bell peppers (red, yellow, or green), julienned',
  '1 teaspoon oregano'],
 'Instructions': ['n a large skillet or wok, stir-fry beef in ginger over medium-high heat until browned.',
  'Add bell peppers and oregano and stir-fry until peppers are crisp-tender.',
  'Serve with rice or noodles.']}